# NLP Challenge

In [36]:
!pip install sentencepiece
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [37]:
import pandas as pd
import numpy as np 
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import sentencepiece as spm
import os 

In [38]:
os.getcwd()

'C:\\Users\\Phenix Labs\\Desktop\\Ansika\\NLP Challange'

In [39]:
df = pd.read_json(r'dataset/train_mr.jsonl', lines=True)
df = pd.json_normalize(df['row'])

In [40]:
df.count()

id        6900
input     6900
target    6900
url       6900
dtype: int64

In [41]:
df.head()

,id,input,target,url
0,1,उपमुख्यमंत्री अजित पवार ः आम्ही सर्वजण उद्धव ठ...,मतभिन्नता असली तरी महाविकास आघाडी मजबूतच - अजि...,https://www.dainikprabhat.com/despite-differen...
1,2,टीम महाराष्ट्र देशा : भारतीय हवाई दलाने नियंत्...,"मुख्यमंत्री आणि पोलीस यांच्यातील बैठक संपली , ...",https://maharashtradesha.com/indian-air-strike...
2,3,सांगली - सांगली शहरातील 100 फुटी रोड परिसरात न...,सांगलीत अज्ञातांकडून वाहनांची जाळपोळ,https://www.dainikprabhat.com/vehicles-burnt-b...
3,4,ऱ्या पावसामुळे महाराष्ट्रावर भीषण दुष्काळाचे ...,जालन्यात पाणी टंचाईने घेतला महिलेचा जीव,https://amnews.live/news/maharashtra/womans-bo...
4,5,मुंबईत मुख्यमंत्री देवेंद्र फडणवीस यांच्या हस्...,लोकशाहीर अण्णाभाऊ साठे यांच्या टपाल तिकिटाचे ग...,https://www.dainikprabhat.com/publication-of-p...


In [42]:
df['words'] = df['input'].str.split()             # split sentemce into words 
all_words = df['words'].explode()                 # create a sreies with all the words 
word_frequencies = all_words.value_counts()       # counting series with frequency of each word and that word  
print('unique words - ',len(word_frequencies))

unique words -  26320


In [43]:
def merge_unicode_files(file1, file2, output_file):
    # Open all files with UTF-8 encoding to handle Unicode text
    with open(file1, 'r', encoding='utf-8') as f1, \
         open(file2, 'r', encoding='utf-8') as f2, \
         open(output_file, 'w', encoding='utf-8') as fout:
        
        # Read contents and write them to output
        fout.write(f1.read())
        fout.write("\n")  # Optional: add a newline between files
        fout.write(f2.read())

    print(f"Files '{file1}' and '{file2}' have been merged into '{output_file}' successfully.")

In [44]:
df['input'].to_csv(r'dataset/input.txt', index=False, header=False)
df['target'].to_csv(r'dataset/output.txt', index=False, header=False)

merge_unicode_files(r"dataset/input.txt", "output.txt", "corpus.txt")

Files 'input.txt' and 'output.txt' have been merged into 'corpus.txt' successfully.


In [45]:
spm.SentencePieceTrainer.train(
    input='corpus.txt',
    model_prefix=r'model/tokenizer/spm_tokenizer',
    pad_id=0,
    unk_id=1,
    bos_id=2,
    eos_id=3,
    vocab_size=800,       #30000                 # You can tune this
    model_type='bpe',                         # or 'unigram'
    character_coverage=0.99995,     #1.0              # For Indic languages use 1.0 to cover all characters
    train_extremely_large_corpus=False
)

In [46]:
sp = spm.SentencePieceProcessor()
sp.load(r'model/tokenizer/spm_tokenizer.model')

#df['tokens_input'] = df['input'].apply(lambda x: sp.encode(x, out_type=str))
df['ids_input'] = df['input'].apply(lambda x: sp.encode(x, out_type=int))

#df['tokens_target'] = df['target'].apply(lambda x: sp.encode(x, out_type=str))
df['ids_target'] = df['target'].apply(lambda x: sp.encode(x, out_type=int))


In [47]:
print("SentencePiece vocab size:", sp.get_piece_size())
print("Max token id in data:", max(max(x) for x in df['ids_input']))

SentencePiece vocab size: 800
Max token id in data: 796


In [48]:
decoded = sp.decode(df['ids_target'][0])
print(df['ids_target'][0])
decoded

[272, 734, 712, 305, 708, 73, 83, 32, 40, 134, 208, 165, 435, 472, 9, 721, 729, 730, 705, 716, 87, 535, 203]


'मतभिन्नता असली तरी महाविकास आघाडी मजबूतच - अजित पवार'

In [49]:
print(df.head())

  id                                              input  \
0  1  उपमुख्यमंत्री अजित पवार ः आम्ही सर्वजण उद्धव ठ...   
1  2  टीम महाराष्ट्र देशा : भारतीय हवाई दलाने नियंत्...   
2  3  सांगली - सांगली शहरातील 100 फुटी रोड परिसरात न...   
3  4  ऱ्या पावसामुळे महाराष्ट्रावर भीषण दुष्काळाचे ...   
4  5  मुंबईत मुख्यमंत्री देवेंद्र फडणवीस यांच्या हस्...   

                                              target  \
0  मतभिन्नता असली तरी महाविकास आघाडी मजबूतच - अजि...   
1  मुख्यमंत्री आणि पोलीस यांच्यातील बैठक संपली , ...   
2               सांगलीत अज्ञातांकडून वाहनांची जाळपोळ   
3            जालन्यात पाणी टंचाईने घेतला महिलेचा जीव   
4  लोकशाहीर अण्णाभाऊ साठे यांच्या टपाल तिकिटाचे ग...   

                                                 url  \
0  https://www.dainikprabhat.com/despite-differen...   
1  https://maharashtradesha.com/indian-air-strike...   
2  https://www.dainikprabhat.com/vehicles-burnt-b...   
3  https://amnews.live/news/maharashtra/womans-bo...   
4  https://www.dainikprabhat

In [50]:
df['ids_input'][0]

[332,
 171,
 191,
 196,
 535,
 203,
 699,
 761,
 224,
 511,
 386,
 721,
 719,
 464,
 257,
 714,
 630,
 353,
 34,
 61,
 713,
 24,
 87,
 724,
 713,
 511,
 386,
 716,
 30,
 719,
 270,
 464,
 257,
 223,
 630,
 353,
 11,
 717,
 722,
 705,
 733]

In [51]:
sp.encode('उपमुख्यमंत्री', out_type=int)

[332, 171, 191, 196]

In [52]:
sp.decode([100])

'प्रति'

In [53]:
df['input'][0]

'उपमुख्यमंत्री अजित पवार ः आम्ही सर्वजण उद्धव ठाकरेंसोबत बारामती -आम्ही सर्वच जण मुख्यमंत्री उद्धव ठाकरे यांच्यासोबत आहोत.'

In [54]:
df['input'][1]

'टीम महाराष्ट्र देशा : भारतीय हवाई दलाने नियंत्रण रेषा ओलांडून जैश ए मोहम्मदला चांगलाच धडा शिकवला आहे.'